In [1]:
%load_ext autoreload
%autoreload 2
# For reloading code such that code changes are taken into account

In [2]:
project_id = 'test-project-221821'

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.environ['HOME'], 'gcp/secret/admin-service-account.json')
import sys
sys.path.append(os.path.join(os.environ['HOME'], 'gcp'))

from cloud_ml.client import CloudML
from cloud_storage.client import CloudStorage
import tensorflow as tf

In [3]:
ml = CloudML(project_id)
cs = CloudStorage(project_id)

Traceback (most recent call last):
  File "/home/vukglisovic/miniconda/envs/google-python-clients/lib/python2.7/site-packages/googleapiclient/discovery_cache/__init__.py", line 41, in autodetect
    from . import file_cache
  File "/home/vukglisovic/miniconda/envs/google-python-clients/lib/python2.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 41, in <module>
    'file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth')
ImportError: file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
INFO:googleapiclient.discovery:URL being requested: GET https://www.googleapis.com/discovery/v1/apis/ml/v1/rest


# Create a model and bucket for storing data

In [4]:
model_name = 'test_model'
description = 'test description'

ml.create_model(model_name, description)

INFO:googleapiclient.discovery:URL being requested: POST https://ml.googleapis.com/v1/projects/test-project-221821/models?alt=json


{u'description': u'test description',
 u'etag': u'fuSwnGye29A=',
 u'name': u'projects/test-project-221821/models/test_model',
 u'regions': [u'us-central1']}

In [10]:
bucket_name = 'vuk-mnist-dataset'
cs.create_bucket(bucket_name)

INFO:root:Created bucket: vuk-mnist-dataset


In [15]:
(Xtrain, ytrain), (Xtest, ytest) = tf.keras.datasets.mnist.load_data()

print(Xtrain.shape, ytrain.shape)
print(Xtest.shape, ytest.shape)

((60000, 28, 28), (60000,))
((10000, 28, 28), (10000,))


# Clean up

In [12]:
ml.delete_model(model_name)

{u'metadata': {u'@type': u'type.googleapis.com/google.cloud.ml.v1.OperationMetadata',
  u'createTime': u'2018-12-08T21:46:54Z',
  u'modelName': u'projects/test-project-221821/models/test_model',
  u'operationType': u'DELETE_MODEL'},
 u'name': u'projects/test-project-221821/operations/delete_model_test_model-1544305614'}

In [16]:
cs.delete_bucket(bucket_name)

INFO:root:Deleted bucket vuk-mnist-dataset
